# Assignment problem - Enumeration


## The problem

There are $m$ workers and an equal number of tasks.
Each worker can perform any task, with a cost that can vary depending on the worker-task assignment.
We want to carry out all the task, assigning only one task to each worker and one task to each worker,
such that the total cost of the assignment is minimised.
The total cost of assigning all tasks is equal to the sum of the costs of assignment for each worker.

## The instance

The number of tasks and workers: $m$
  
The cost to assign to the $i^{\text{th}}$ worker the $j^{\text{th}}$ task
$c_{ij},\text{ for all }\ i=1,2,\ldots,m\text{ and }j=1,2,\ldots,m$

Consider the following instance
  
  Number of workers and tasks: $m \gets 4$
  
  Assignment costs:
  
  | Worker | Task 1 | Task 2 | Task 3 | Task 4 |
  |--------|--------|--------|--------|--------|
  |    1   |    14  |    5   |    8   |    7   |
  |    2   |     2  |   12   |    6   |    5   |
  |    3   |     7  |    8   |    3   |    9   |
  |    4   |     2  |    4   |    6   |   10   |
  

In [43]:
# Assignment problem: Finding the optimal assignment of workers to jobs with minimum total cost.

import itertools

# Instance definition:
# m: Number of workers (and jobs, as it's a square assignment problem).
# c: Cost matrix where c[i][j] represents the cost of assigning worker i to job j.
m = 4
c = [[14,  5,  8,  7],
     [ 2, 12,  6,  5],
     [ 7,  8,  3,  9],
     [ 2,  4,  6, 10]]

def total_cost(solution):
    """
    Calculates the total cost of a given assignment solution.

    Args:
        solution: A list representing the assignment. solution[i] indicates the job assigned to worker i.
                  For example, [1, 0, 3, 2] means worker 0 is assigned job 1, worker 1 is assigned job 0, etc.

    Returns:
        The total cost of the assignment.
    """
    total = 0  # Initialize the accumulator for the total cost.
    for worker_index, job_index in enumerate(solution):
        # worker_index: Index of the worker (0 to m-1).
        # job_index: Index of the job assigned to the worker.
        total += c[worker_index][job_index]  # Add the cost of this assignment to the total.
    return total

# Generate all possible assignment solutions:
# feasible_solutions: A list of all permutations of job assignments. Each permutation represents a possible solution.
# itertools.permutations(range(0, m)) generates all possible ways to assign jobs (0 to m-1) to workers.
feasible_solutions = list(itertools.permutations(range(0, m)))

def find_min_total_cost(feasible_solutions, total_cost):
    """
    Finds the minimum total cost and the corresponding assignment solution by enumerating all feasible solutions.

    Args:
        feasible_solutions: A list of all possible assignment solutions (permutations).
        total_cost: A function that calculates the total cost of a given solution.

    Returns:
        A tuple containing:
            - The minimum total cost found.
            - The assignment solution that achieves the minimum cost.
    """
    min_cost = float('inf')  # Initialize the minimum cost to infinity.
    min_solution = None  # Initialize the minimum solution to None.

    # Iterate through all feasible solutions:
    for index, solution in enumerate(feasible_solutions):
        cost = total_cost(solution)  # Calculate the cost of the current solution.
        str = f"Exploring the solution number {(index+1):2}: Cost of {solution} is {cost}"

        # Check if the current solution has a lower cost than the current minimum:
        if cost < min_cost:
            min_cost = cost  # Update the minimum cost.
            min_solution = solution  # Update the minimum solution.
            str += " <-- New minimum (incumbent solution)"

        print(str)

    return min_cost, min_solution  # Return the minimum cost and the corresponding solution.

# Find and print the optimal solution:
min_cost, optimal_solution = find_min_total_cost(feasible_solutions, total_cost)
print(f"\nMinimum total cost: {min_cost}")
print(f"Optimal solution:")
for worker_index, job_index in enumerate(optimal_solution):
    print(f"Worker {worker_index + 1} is assigned to Task {job_index + 1}")

Exploring the solution number  1: Cost of (0, 1, 2, 3) is 39 <-- New minimum (incumbent solution)
Exploring the solution number  2: Cost of (0, 1, 3, 2) is 41
Exploring the solution number  3: Cost of (0, 2, 1, 3) is 38 <-- New minimum (incumbent solution)
Exploring the solution number  4: Cost of (0, 2, 3, 1) is 33 <-- New minimum (incumbent solution)
Exploring the solution number  5: Cost of (0, 3, 1, 2) is 33
Exploring the solution number  6: Cost of (0, 3, 2, 1) is 26 <-- New minimum (incumbent solution)
Exploring the solution number  7: Cost of (1, 0, 2, 3) is 20 <-- New minimum (incumbent solution)
Exploring the solution number  8: Cost of (1, 0, 3, 2) is 22
Exploring the solution number  9: Cost of (1, 2, 0, 3) is 28
Exploring the solution number 10: Cost of (1, 2, 3, 0) is 22
Exploring the solution number 11: Cost of (1, 3, 0, 2) is 23
Exploring the solution number 12: Cost of (1, 3, 2, 0) is 15 <-- New minimum (incumbent solution)
Exploring the solution number 13: Cost of (2, 

 ## AMPL code


In [44]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    license_uuid="0ca63f63-acf5-4b2f-bb96-3be37cfd1b7d",
    modules=["cbc"])

Licensed to AMPL Community Edition License for <gionata.massi@gmail.com>.


In [45]:
%%writefile assignment.mod
set Resources;
set Tasks;
set Links := Resources cross Tasks;

param cost {(i, j) in Links} >= 0;
var x {(i, j) in Links} >= 0; # binary;

check: card(Resources) = card(Tasks);

minimize total_cost:
  sum{(i, j) in Links} cost[i, j] * x[i, j];

s.t. one_task_per_resource {i in Resources}:
  sum{(i, j) in Links} x[i, j] = 1;

s.t. one_resourse_per_task {j in Tasks}:
  sum{(i, j) in Links} x[i, j] = 1;


Overwriting assignment.mod


In [46]:
%%writefile assignment.dat
set Resources := R1 R2 R3 R4;
set Tasks := T1 T2 T3 T4;

param cost :
       T1 T2 T3 T4 =
   R1  14  5  8  7
   R2   2 12  6  5
   R3   7  8  3  9
   R4   2  4  6 10;

Overwriting assignment.dat


In [47]:
ampl.read('assignment.mod')
ampl.eval('show;')


parameter:   cost

sets:   Links   Resources   Tasks

variable:   x

constraints:   one_resourse_per_task   one_task_per_resource

objective:   total_cost

checks: one, called check 1.


In [48]:
ampl.readData('assignment.dat')
ampl.eval('expand;')

minimize total_cost:
	14*x['R1','T1'] + 5*x['R1','T2'] + 8*x['R1','T3'] + 7*x['R1','T4'] + 
	2*x['R2','T1'] + 12*x['R2','T2'] + 6*x['R2','T3'] + 5*x['R2','T4'] + 
	7*x['R3','T1'] + 8*x['R3','T2'] + 3*x['R3','T3'] + 9*x['R3','T4'] + 
	2*x['R4','T1'] + 4*x['R4','T2'] + 6*x['R4','T3'] + 10*x['R4','T4'];

subject to one_task_per_resource['R1']:
	x['R1','T1'] + x['R1','T2'] + x['R1','T3'] + x['R1','T4'] = 1;

subject to one_task_per_resource['R2']:
	x['R2','T1'] + x['R2','T2'] + x['R2','T3'] + x['R2','T4'] = 1;

subject to one_task_per_resource['R3']:
	x['R3','T1'] + x['R3','T2'] + x['R3','T3'] + x['R3','T4'] = 1;

subject to one_task_per_resource['R4']:
	x['R4','T1'] + x['R4','T2'] + x['R4','T3'] + x['R4','T4'] = 1;

subject to one_resourse_per_task['T1']:
	x['R1','T1'] + x['R2','T1'] + x['R3','T1'] + x['R4','T1'] = 1;

subject to one_resourse_per_task['T2']:
	x['R1','T2'] + x['R2','T2'] + x['R3','T2'] + x['R4','T2'] = 1;

subject to one_resourse_per_task['T3']:
	x['R1','T3'] + x['R2','T3'

In [49]:
ampl.eval("expand one_resourse_per_task['T1'];")

subject to one_resourse_per_task['T1']:
	x['R1','T1'] + x['R2','T1'] + x['R3','T1'] + x['R4','T1'] = 1;



In [50]:
ampl.eval("option solver cbc;")
ampl.solve()
ampl.display('total_cost')
ampl.display('{(i, j) in Links : x[i, j] = 1} x[i, j]')

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 15
0 simplex iterations
total_cost = 15

x[i,j] :=
R1 T2   1
R2 T4   1
R3 T3   1
R4 T1   1
;



## Exercise 1

Solve the problem for

Number of workers and tasks: $m \gets 4$
  
Assignment costs:
  
  | Worker | Task 1 | Task 2 | Task 3 | Task 4 |
  |--------|--------|--------|--------|--------|
  |    1   |    22  |   18   |   30   |   18   |
  |    2   |    18  |   --   |   27   |   22   |
  |    3   |    16  |   22   |   --   |   14   |
  |    4   |    21  |   --   |   25   |   28   |


- How to tackle the unfeasible assignment (written as --)?
- Write down the mathematical model for the instance in the previous slides.
- Generate the feasible solutions and determine the optimal solution
- How much feasible solutions?

## AMPL solution

In [51]:
%%writefile assignment_es1.mod
set Resources;
set Tasks;
set Links within Resources cross Tasks;

param cost {(i, j) in Links} >= 0;
var x {(i, j) in Links} >= 0; # binary;

check: card(Resources) = card(Tasks);

minimize total_cost:
  sum{(i, j) in Links} cost[i, j] * x[i, j];

s.t. one_task_per_resource {i in Resources}:
  sum{(i, j) in Links} x[i, j] = 1;

s.t. one_resourse_per_task {j in Tasks}:
  sum{(i, j) in Links} x[i, j] = 1;


Overwriting assignment_es1.mod


In [52]:
ampl.reset()
ampl.read('assignment_es1.mod')
ampl.eval('show;')
ampl.set['Resources'] = ['R1', 'R2', 'R3', 'R4']
ampl.set['Tasks'] = ['T1', 'T2', 'T3', 'T4']
ampl.set['Links'] = [('R1', 'T1'), ('R1', 'T2'), ('R1', 'T3'), ('R1', 'T4'),
                     ('R2', 'T1'),               ('R2', 'T3'),  ('R2', 'T4'),
                     ('R3', 'T1'),  ('R3', 'T2'),               ('R3', 'T4'),
                     ('R4', 'T1'),               ('R4', 'T3'),  ('R4', 'T4')]
ampl.param['cost'] = [22, 18, 30, 18,
                      18,           27, 22,
                      16,  22,           14,
                      21,           25, 28]
ampl.eval('expand;')
ampl.solve(solver="cbc")
ampl.display('total_cost')
ampl.display('{(i, j) in Links : x[i, j] > 0} x[i, j]')



parameter:   cost

sets:   Links   Resources   Tasks

variable:   x

constraints:   one_resourse_per_task   one_task_per_resource

objective:   total_cost

checks: one, called check 1.
minimize total_cost:
	22*x['R1','T1'] + 18*x['R1','T2'] + 30*x['R1','T3'] + 18*x['R1','T4']
	 + 18*x['R2','T1'] + 27*x['R2','T3'] + 22*x['R2','T4'] + 
	16*x['R3','T1'] + 22*x['R3','T2'] + 14*x['R3','T4'] + 21*x['R4','T1']
	 + 25*x['R4','T3'] + 28*x['R4','T4'];

subject to one_task_per_resource['R1']:
	x['R1','T1'] + x['R1','T2'] + x['R1','T3'] + x['R1','T4'] = 1;

subject to one_task_per_resource['R2']:
	x['R2','T1'] + x['R2','T3'] + x['R2','T4'] = 1;

subject to one_task_per_resource['R3']:
	x['R3','T1'] + x['R3','T2'] + x['R3','T4'] = 1;

subject to one_task_per_resource['R4']:
	x['R4','T1'] + x['R4','T3'] + x['R4','T4'] = 1;

subject to one_resourse_per_task['T1']:
	x['R1','T1'] + x['R2','T1'] + x['R3','T1'] + x['R4','T1'] = 1;

subject to one_resourse_per_task['T2']:
	x['R1','T2'] + x['R3','T2'] = 1

## Exercise 2

Number of workers and tasks: $m \gets 10$
  
Assignment costs:

| Worker | Task 1 | Task 2 | Task 3 | Task 4 | Task 5 | Task 6 | Task 7 | Task 8 | Task 9 | Task 10 |
|---|---|---|---|---|---|---|---|---|---|---|
| 1 | 45 | 12 | 87 | 23 | 66 | 91 | 5 | 38 | 72 | 19 |
| 2 | 8 | 55 | 31 | 98 | 15 | 42 | 77 | 60 | 29 | 84 |
| 3 | 93 | 27 | 64 | 10 | 81 | 36 | 58 | 7 | 49 | 22 |
| 4 | 17 | 79 | 50 | 48 | 9 | 73 | 25 | 95 | 13 | 68 |
| 5 | 62 | 3 | 90 | 59 | 41 | 18 | 88 | 33 | 76 | 4 |
| 6 | 21 | 86 | 14 | 75 | 30 | 97 | 53 | 6 | 82 | 39 |
| 7 | 70 | 44 | 28 | 83 | 65 | 2 | 99 | 47 | 11 | 56 |
| 8 | 35 | 92 | 67 | 20 | 89 | 51 | 16 | 78 | 43 | 26 |
| 9 | 57 | 1 | 40 | 94 | 24 | 71 | 63 | 80 | 37 | 52 |
| 10 | 46 | 69 | 34 | 12 | 54 | 96 | 29 | 85 | 61 | 74 |

- Generate the feasible solutions and determine the/an optimal solution.
- How much feasible solutions?

## AMPL solution

In [53]:
ampl.reset()
ampl.read('assignment.mod')
ampl.set['Resources'] = range(1, 11)
ampl.set['Tasks'] = range(1, 11)
ampl.param['cost'] = [
 45, 12, 87, 23, 66, 91, 5, 38, 72, 19,
 8, 55, 31, 98, 15, 42, 77, 60, 29, 84,
 93, 27, 64, 10, 81, 36, 58, 7, 49, 22,
 17, 79, 50, 48, 9, 73, 25, 95, 13, 68,
 62, 3, 90, 59, 41, 18, 88, 33, 76, 4,
 21, 86, 14, 75, 30, 97, 53, 6, 82, 39,
 70, 44, 28, 83, 65, 2, 99, 47, 11, 56,
 35, 92, 67, 20, 89, 51, 16, 78, 43, 26,
 57, 1, 40, 94, 24, 71, 63, 80, 37, 52,
 46, 69, 34, 12, 54, 96, 29, 85, 61, 74
]
ampl.eval('expand;')
ampl.solve()
ampl.display('total_cost')
ampl.display('{(i, j) in Links : x[i, j] = 1} x[i, j]')


minimize total_cost:
	45*x[1,1] + 12*x[1,2] + 87*x[1,3] + 23*x[1,4] + 66*x[1,5] + 91*x[1,6]
	 + 5*x[1,7] + 38*x[1,8] + 72*x[1,9] + 19*x[1,10] + 8*x[2,1] + 55*x[2,2]
	 + 31*x[2,3] + 98*x[2,4] + 15*x[2,5] + 42*x[2,6] + 77*x[2,7] + 
	60*x[2,8] + 29*x[2,9] + 84*x[2,10] + 93*x[3,1] + 27*x[3,2] + 64*x[3,3]
	 + 10*x[3,4] + 81*x[3,5] + 36*x[3,6] + 58*x[3,7] + 7*x[3,8] + 49*x[3,9]
	 + 22*x[3,10] + 17*x[4,1] + 79*x[4,2] + 50*x[4,3] + 48*x[4,4] + 
	9*x[4,5] + 73*x[4,6] + 25*x[4,7] + 95*x[4,8] + 13*x[4,9] + 68*x[4,10]
	 + 62*x[5,1] + 3*x[5,2] + 90*x[5,3] + 59*x[5,4] + 41*x[5,5] + 18*x[5,6]
	 + 88*x[5,7] + 33*x[5,8] + 76*x[5,9] + 4*x[5,10] + 21*x[6,1] + 
	86*x[6,2] + 14*x[6,3] + 75*x[6,4] + 30*x[6,5] + 97*x[6,6] + 53*x[6,7]
	 + 6*x[6,8] + 82*x[6,9] + 39*x[6,10] + 70*x[7,1] + 44*x[7,2] + 
	28*x[7,3] + 83*x[7,4] + 65*x[7,5] + 2*x[7,6] + 99*x[7,7] + 47*x[7,8] + 
	11*x[7,9] + 56*x[7,10] + 35*x[8,1] + 92*x[8,2] + 67*x[8,3] + 20*x[8,4]
	 + 89*x[8,5] + 51*x[8,6] + 16*x[8,7] + 78*x[8,8] + 43*x[8,9] + 
	26*